In [1]:
%run imports_and_functions.ipynb

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Jupyter-Gmaps Copyright

In [18]:
# Connection with yelp api managed through github sourced yelpapi, author gfairchild, permitted with
# display of copyright notice below.
"""
    Copyright (c) 2013, Los Alamos National Security, LLC
    All rights reserved.
    Redistribution and use in source and binary forms, with or without modification, are permitted provided that the
    following conditions are met:
    * Redistributions of source code must retain the above copyright notice, this list of conditions and the following
      disclaimer.
    * Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the
      following disclaimer in the documentation and/or other materials provided with the distribution.
    * Neither the name of Los Alamos National Security, LLC nor the names of its contributors may be used to endorse or
      promote products derived from this software without specific prior written permission.
    THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES,
    INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
    DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
    SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
    SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
    WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
    OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
"""

'\n    Copyright (c) 2013, Los Alamos National Security, LLC\n    All rights reserved.\n    Redistribution and use in source and binary forms, with or without modification, are permitted provided that the\n    following conditions are met:\n    * Redistributions of source code must retain the above copyright notice, this list of conditions and the following\n      disclaimer.\n    * Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the\n      following disclaimer in the documentation and/or other materials provided with the distribution.\n    * Neither the name of Los Alamos National Security, LLC nor the names of its contributors may be used to endorse or\n      promote products derived from this software without specific prior written permission.\n    THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES,\n    INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHAN

## Global Variables

In [2]:
from yelpapi import YelpAPI

API_KEY1 = "G1-37Mu0IHzYjviLF5gdpXJ_GdNZhKBCVuVLCfwJ5oRLivMCKpWrQMYTurfGuihavP0K7gOiLpH-0Ze75L96RjthUEoMukghlMqexUaziQ4agkeu6MfdtW5enpgvXHYx"
API_KEY2 = "fKGT-oiIvnk4B4rtfQsKTzQerIG6rtyC-qthL6COtUYxFJuioNn_sn_ty6l3FlIZidNg0uBZBvgwN4kudHJ4GtiQoOpcEu3U11Is0yI5neN7y1kTk0KKOuoqtMkvXHYx"

yelp_api = YelpAPI(API_KEY1)

## Extracting Area Data from Yelp

### Storing data for one Yelp Search

In [3]:
class SearchData(object) :
    
    # Search parameters must always be given a location, for this project always a street address
    # For one location and one main_category of business
    
    def __init__(self, name, main_category, location=None, coords=(90,90), 
                 limit = 50, offset = 0, radius=search_radius, df=None) :
        self.name = name
        self.main_category = main_category
        self.coords = coords
        self.location = location # assume input of either text location and if not lat/long coords
        self.limit = limit
        self.offset = offset
        self.radius = radius  # radius (meters) to search within, yelp sometimes grabs outside the radius
        if df == None :
            self.df = pd.DataFrame(columns = features)
        else : self.df = df
        self.sort_by = 'distance'

    # other optional parameters
    #     term    # user inputted search term
    #     sort_by # default is best_match, other options include distance and review_count but these may 
    # limit max search results returned to only 40, per yelp documentation

    def printparams(self) :
        print('location: ', self.location, ', coords: ', self.coords, ', offset: ', self.offset, 
        ', radius: ', self.radius, ', main_category: ', self.main_category)
        print('Dataframe size:', self.df.shape)
    
    def add50rows(self) :
        latlong = None
        if self.location == None :  
            lat, long = self.coords
            latlong = str(lat) + ', ' + str(long)
            location = latlong
        else : location = self.location
        search_results = yelp_api.search_query(location=location, 
                                               limit=self.limit, offset=self.offset,
                                               categories=self.main_category, sort_by=self.sort_by)
        prev_count = self.df.shape[0]
        max_distance = 0
        for business in search_results['businesses'] :
            self.df = self.df.append({tupl[0] : tupl[1] for tupl in business.items() if tupl[0] in features},
                                    ignore_index=True, sort=True)
            if  business['distance'] > max_distance : max_distance = business['distance']
#         print(self.df.shape[0] - prev_count, 'rows added to', self.name, ' for ', self.main_category)
        return max_distance
    
    def addyelpdata(self) :
        end = False
        max_distance = 0
        while (self.offset < 1000) & (not end) :
            try :
                max_distance = self.add50rows()
                if max_distance > self.radius : end = True
                self.offset += self.limit
            except : # usually this means at end of yelp search results or hit 1000 result hard limit
                end = True
        
        self.df['latitude'] = self.df['coordinates'].map((lambda x : x['latitude']))
        self.df['longitude'] = self.df['coordinates'].map((lambda x : x['longitude']))
        self.df['main_category'] = self.main_category
#         print(self.main_category, ' done, size: ', self.df.shape, ' max_dist = ', max_distance)

### Functions for One City Searches for Every Business Category 

In [4]:
class AreaData(object) :

    def __init__(self, name, coords=(90,90), location=None, radius=search_radius,
                df=pd.DataFrame()) :
        self.name = name
        self.coords = coords
        self.location = location
        self.radius = radius  # radius (meters) to search within, yelp sometimes grabs outside the radius
        self.df = df
        self.searches = []

    def extract_yelp_businesses(self, location=None, coords=(90,90)) :
        for category in main_alias :
            search = SearchData('boston1', category, location=location, coords=coords, radius=self.radius)
            search.addyelpdata()
            self.df = self.df.append(search.df, ignore_index = True, sort=True)
            self.searches.append(search)
        return self.df

In [26]:
def yelp_extra_square_search(main_category, lat, long, df=pd.DataFrame()) :
    lat1000m = -115.161605-(-115.172713)
    scale = 5
    corner1 = (lat-lat1000m*scale, long-lat1000m*scale)
    corner2 = (lat+lat1000m*scale, long-lat1000m*scale)
    corner3 = (lat+lat1000m*scale, long+lat1000m*scale)
    corner4 = (lat-lat1000m*scale, long+lat1000m*scale)
    top = lat+lat1000m*scale, long
    bottom = lat-lat1000m*scale, long
    left = lat, long-lat1000m*scale
    right = lat, long+lat1000m*scale
    coordlist = [corner1, left, corner2, top, corner3, right, corner4, bottom]
    for coord in coordlist :
        search = SearchData('subsets', main_category, coords=coord, radius=1000*scale*0.7)
        search.addyelpdata()
        df = df.append(search.df, ignore_index = True, sort=True)
    display = df.shape
    dropmask = df[['alias','latitude','longitude','review_count']].duplicated(keep='first')
    df = df[~dropmask]
#     print(display, ' dropped to ', df.shape)
    return df

In [7]:
# boston1 = AreaData('Boston1', coords=(startlat, startlong), radius=maxradius)
# df = boston1.extract_yelp_businesses(coords=(testlat, testlong))
# df.shape

active  done, size:  (800, 12)  max_dist =  5067.800938992921
arts  done, size:  (600, 12)  max_dist =  6190.962542694475
automotive  done, size:  (500, 12)  max_dist =  5266.645117196608
beautysvc  done, size:  (1000, 12)  max_dist =  4512.937708552031
education  done, size:  (450, 12)  max_dist =  5130.022659200853
eventservices  done, size:  (800, 12)  max_dist =  5058.790089764473
financialservices  done, size:  (400, 12)  max_dist =  5268.37530991864
food  done, size:  (1000, 12)  max_dist =  3339.691995487489
health  done, size:  (1000, 12)  max_dist =  3035.3559560361236
homeservices  done, size:  (1000, 12)  max_dist =  3078.7656519261473
hotelstravel  done, size:  (700, 12)  max_dist =  5329.777458536324
localservices  done, size:  (1000, 12)  max_dist =  4309.098729159692
massmedia  done, size:  (88, 12)  max_dist =  11352.053314398836
nightlife  done, size:  (750, 12)  max_dist =  5266.345251586332
pets  done, size:  (200, 12)  max_dist =  5746.45478686026
professional  done

(14538, 12)